# 第四章 构建RAG应用

## 4.1 将LLM 接入 LangChain

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

### 4.1.1 基于 LangChain 调用 ChatGPT

LangChain 提供了对于多种大模型的封装，基于 LangChain 的接口可以便捷地调用 ChatGPT 并将其集合在以 LangChain 为基础框架搭建的个人应用中。我们在此简述如何使用 LangChain 接口来调用 ChatGPT。

注意，基于 LangChain 接口调用 ChatGPT 同样需要配置你的个人密钥，配置方法同上。

从 `langchain.chat_models` 导入 `OpenAI` 的对话模型 `ChatOpenAI` 。 除去OpenAI以外，`langchain.chat_models` 还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.chat_models)。

In [13]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv(),override=True)

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

没有安装 langchain-openai 的话，请先运行下面进行代码！

In [3]:
from langchain_openai import ChatOpenAI

接下来你需要实例化一个 ChatOpenAI 类，可以在实例化时传入超参数来控制回答，例如 `temperature` 参数。

In [14]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
import os
model = os.environ.get("MODEL_NAME")
llm = ChatOpenAI(temperature=0.0,model=model)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000233F1A17890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000233F3B2A090>, root_client=<openai.OpenAI object at 0x00000233F3B2B1D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000233F1A147D0>, model_name='MiniMax-M2.1', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

上面的 cell 假设你的 OpenAI API 密钥是在环境变量中设置的，如果您希望手动指定API密钥，请使用以下代码：

In [ ]:
# llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")

可以看到，默认调用的是 ChatGPT-3.5 模型。另外，几种常用的超参数设置包括：

    · model_name：所要使用的模型，默认为 ‘gpt-3.5-turbo’，参数设置与 OpenAI 原生接口参数设置一致。

    · temperature：温度系数，取值同原生接口。

    · openai_api_key：OpenAI API key，如果不使用环境变量设置 API Key，也可以在实例化时设置。

    · openai_proxy：设置代理，如果不使用环境变量设置代理，也可以在实例化时设置。

    · streaming：是否使用流式传输，即逐字输出模型回答，默认为 False，此处不赘述。

    · max_tokens：模型输出的最大 token 数，意义及取值同上。

当我们初始化了你选择的`LLM`后，我们就可以尝试使用它！让我们问一下“请你自我介绍一下自己！”

In [8]:
output = llm.invoke("请你自我介绍一下自己！")

In [ ]:
output

AIMessage(content='你好，我是一个智能助手，专门为您提供各种服务和帮助。我可以回答您的问题，提供信息和建议，帮助您解决问题。如果您有任何需要，请随时告诉我，我会尽力帮助您。感谢您选择我作为您的助手！如果您有任何问题，请随时问我。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 20, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c611b32a-4adf-47af-9b97-6dda68a117e1-0', usage_metadata={'input_tokens': 20, 'output_tokens': 109, 'total_tokens': 129})

在我们开发大模型应用时，大多数情况下不会直接将用户的输入直接传递给 LLM。通常，他们会将用户输入添加到一个较大的文本中，称为`提示模板`，该文本提供有关当前特定任务的附加上下文。
PromptTemplates 正是帮助解决这个问题！它们捆绑了从用户输入到完全格式化的提示的所有逻辑。这可以非常简单地开始 - 例如，生成上述字符串的提示就是：


我们需要先构造一个个性化 Template：

In [9]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

接下来让我们看一下构造好的完整的提示模版：

In [10]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

我们知道聊天模型的接口是基于消息（message），而不是原始的文本。PromptTemplates 也可以用于产生消息列表，在这种样例中，`prompt`不仅包含了输入内容信息，也包含了每条`message`的信息(角色、在列表中的位置等)。通常情况下，一个 `ChatPromptTemplate` 是一个 `ChatMessageTemplate` 的列表。每个 `ChatMessageTemplate` 包含格式化该聊天消息的说明（其角色以及内容）。

让我们一起看一个示例：

In [15]:
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})])

接下来让我们调用定义好的`llm`和`messages`来输出回答：

In [16]:
output  = llm.invoke(messages)
output

AIMessage(content='<think>\nThe user is asking me to translate a Chinese sentence into English. The sentence appears to be poetic or lyrical in nature. Let me break it down:\n\n我 (I) 带着 (carrying) 比身体重的行李 (luggage heavier than my body), 游入 (swim into) 尼罗河底 (the bottom of the Nile River), 经过几道闪电 (passing through several lightning bolts), 看到 (see) 一堆光圈 (a bunch of halos/light circles), 不确定是不是这里 (not sure if this is the right place).\n\nThis seems to be lyrics from a song. The translation should preserve the poetic nature while making it sound natural in English.\n\nHere\'s my translation:\n\n"I swim into the depths of the Nile, carrying luggage heavier than my body, passing through several bolts of lightning, seeing a cluster of light circles, not sure if this is the place."\n\nOr more poetically:\n\n"Carrying luggage heavier than my body, I swim into the depths of the Nile, passing through flashes of lightning, glimpsing a halo of light—unsure if this is where I belong."\n\nLet me provi

OutputParsers 将语言模型的原始输出转换为可以在下游使用的格式。 OutputParsers 有几种主要类型，包括：
- 将 LLM 文本转换为结构化信息（例如 JSON） 
- 将 ChatMessage 转换为字符串 
- 将除消息之外的调用返回的额外信息（如 OpenAI 函数调用）转换为字符串

最后，我们将模型输出传递给 `output_parser`，它是一个 `BaseOutputParser`，这意味着它接受**字符串或 BaseMessage 作为输入**。 StrOutputParser 特别简单地将任何输入转换为字符串。

In [17]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'<think>\nThe user is asking me to translate a Chinese sentence into English. The sentence appears to be poetic or lyrical in nature. Let me break it down:\n\n我 (I) 带着 (carrying) 比身体重的行李 (luggage heavier than my body), 游入 (swim into) 尼罗河底 (the bottom of the Nile River), 经过几道闪电 (passing through several lightning bolts), 看到 (see) 一堆光圈 (a bunch of halos/light circles), 不确定是不是这里 (not sure if this is the right place).\n\nThis seems to be lyrics from a song. The translation should preserve the poetic nature while making it sound natural in English.\n\nHere\'s my translation:\n\n"I swim into the depths of the Nile, carrying luggage heavier than my body, passing through several bolts of lightning, seeing a cluster of light circles, not sure if this is the place."\n\nOr more poetically:\n\n"Carrying luggage heavier than my body, I swim into the depths of the Nile, passing through flashes of lightning, glimpsing a halo of light—unsure if this is where I belong."\n\nLet me provide a clean, accura

从上面结果可以看到，我们通过输出解析器成功将 `ChatMessage` 类型的输出解析为了`字符串`

我们现在可以将所有这些组合成一条链。该链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给语言模型，然后通过（可选）输出解析器传递输出。接下来我们将使用LCEL这种语法去快速实现一条链（chain）。让我们看看它的实际效果！

In [18]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})


'<think>\nThe user is asking me to translate a Chinese sentence into English. Let me analyze the content:\n\n这句话来自周杰伦的歌词，是《稻香》的一部分。让我来准确翻译它。\n\nLet me break it down:\n- "我带着比身体重的行李" - I carry luggage heavier than my body\n- "游入尼罗河底" - swim into the depths of the Nile River\n- "经过几道闪电" - passing through several lightning bolts\n- "看到一堆光圈" - see a bunch of halos/rings of light\n- "不确定是不是这里" - not sure if this is the place\n\nThis appears to be a poetic/lyrical translation. I should provide a translation that captures the imagery and poetic nature of the original text.\n\nLet me provide a translation that maintains the poetic quality:\n</think>\n\nHere is the English translation:\n\n"I carry luggage heavier than my body, diving into the depths of the Nile River. Passing through several bolts of lightning, I see a cluster of rings of light — not sure if this is the place."\n\n---\n\n**Note:** This lyric is from Jay Chou\'s song "Rice Fragrance" (稻香). The translation preserves the surreal, 

再测试一个样例：

In [19]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'<think>\nThe user wants me to translate this English text into Chinese. Let me break down the sentence:\n\n"I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place."\n\nLet me translate this carefully:\n\n- "I carried luggage heavier than my body" - 我携带着比我的身体还重的行李\n- "and dived into the bottom of the Nile River" - 潜入了尼罗河河底\n- "After passing through several flashes of lightning" - 穿过几道闪电之后\n- "I saw a pile of halos" - 我看到了一圈光环\n- "not sure if this is the place" - 不确定这是不是那个地方\n\nThis appears to be a poetic or metaphorical description, possibly from a dream or imaginative narrative. Let me provide a natural Chinese translation.\n</think>\n\n我携带着比自身体重还沉的行李，潜入了尼罗河的河底。穿过几道闪电之后，我看到了一圈光环，不确定这里是不是那个地方。\n\n---\n\n**翻译说明：**\n- "carried luggage heavier than my body" - 译为"携带着比自身体重还沉的行李"，表达出一种沉重、超负荷的感觉\n- "the bottom of the Nile River" - 译为"尼罗河的河底"，保持地理专有名词的准确性\n- "a 

> 什么是 LCEL ？ 
LCEL（LangChain Expression Language，Langchain的表达式语言），LCEL是一种新的语法，是LangChain工具包的重要补充，他有许多优点，使得我们处理LangChain和代理更加简单方便。

- LCEL提供了异步、批处理和流处理支持，使代码可以快速在不同服务器中移植。
- LCEL拥有后备措施，解决LLM格式输出的问题。
- LCEL增加了LLM的并行性，提高了效率。
- LCEL内置了日志记录，即使代理变得复杂，有助于理解复杂链条和代理的运行情况。

用法示例：

`chain = prompt | model | output_parser`

上面代码中我们使用 LCEL 将不同的组件拼凑成一个链，在此链中，用户输入传递到提示模板，然后提示模板输出传递到模型，然后模型输出传递到输出解析器。| 的符号类似于 Unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为下一个组件的输入。


### 4.1.2 使用 LangChain 调用百度文心一言

我们同样可以通过 LangChain 框架来调用百度文心大模型，以将文心模型接入到我们的应用框架中。

In [ ]:

from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]


In [ ]:
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint

llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

[WARNING][2025-03-05 19:41:13.652] redis_rate_limiter.py:21 [t:8258539328]: No redis installed, RedisRateLimiter unavailable. Ignore this warning if you don't need to use qianfan SDK in distribution environment
/var/folders/yd/c4q_f88j1l70g7_jcb6pdnb80000gn/T/ipykernel_51550/1209153611.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm("你好，请你自我介绍一下！")
[ERROR][2025-03-05 19:41:13.835] base.py:134 [t:8258539328]: http request url https://qianfan.baidubce.com/wenxinworkshop/service/list failed with http status code 403
error code from baidu: IamSignatureInvalid
error message from baidu: IamSignatureInvalid, cause: Could not find credential.
request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Host': 'qianfan.baidubce.com', 'request-source': '

你好！我是一个人工智能语言模型，我的名字是文心一言。我能够与人进行自然语言交互，并提供各种信息和服务。如果你有任何问题或需要帮助，请随时告诉我，我会尽力为你提供帮助。


### 4.1.3 讯飞星火

我们同样可以通过 LangChain 框架来调用讯飞星火大模型，更多信息参考[SparkLLM](https://python.langchain.com/docs/integrations/llms/sparkllm)

我们希望像调用 ChatGPT 那样直接将秘钥存储在 .env 文件中，并将其加载到环境变量，从而隐藏秘钥的具体细节，保证安全性。因此，我们需要在 .env 文件中配置`IFLYTEK_SPARK_APP_ID`、 `IFLYTEK_SPARK_API_KEY` 和 `IFLYTEK_SPARK_API_SECRET`，并使用以下代码加载：

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
IFLYTEK_SPARK_APP_ID = os.environ["IFLYTEK_SPARK_APP_ID"]
IFLYTEK_SPARK_API_KEY = os.environ["IFLYTEK_SPARK_API_KEY"]
IFLYTEK_SPARK_API_SECRET = os.environ["IFLYTEK_SPARK_API_SECRET"]

另外星火各模型对应的`spark_api_url`与`spark_llm_domain`均不相同，可以参考[接口说明](https://www.xfyun.cn/doc/spark/Web.html#_1-%E6%8E%A5%E5%8F%A3%E8%AF%B4%E6%98%8E)选择调用。

In [ ]:
from langchain_community.llms.sparkllm import SparkLLM

# Load the model
llm = SparkLLM(
    model='Spark4.0 Ultra',
    app_id=IFLYTEK_SPARK_APP_ID,
    api_key=IFLYTEK_SPARK_API_KEY,
    api_secret=IFLYTEK_SPARK_API_SECRET,
    spark_api_url="wss://spark-api.xf-yun.com/v4.0/chat",
    spark_llm_domain="4.0Ultra"
    )

In [ ]:
res = llm.invoke("你好，请你自我介绍一下！")
print(res)

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。


从而我们可以将星火大模型加入到 LangChain 架构中，实现在应用中对文心大模型的调用。

### 4.1.4 使用 LangChain 调用智谱 GLM

我们同样可以通过 LangChain 框架来调用智谱 AI 大模型，以将其接入到我们的应用框架中。由于 langchain 中提供的[ChatGLM](https://python.langchain.com/docs/integrations/llms/chatglm)已不可用，因此我们需要自定义一个LLM。

如果你使用的是智谱 GLM API，你需要将我们封装的代码[zhipuai_llm.py](./zhipuai_llm.py)下载到本 Notebook 的同级目录下，才可以运行下列代码来在 LangChain 中使用 GLM。

根据智谱官方宣布以下模型即将弃用，在这些模型弃用后，会将它们自动路由至新的模型。请用户注意在弃用日期之前，将您的模型编码更新为最新版本，以确保服务的顺畅过渡，更多模型相关信息请访问[model](https://open.bigmodel.cn/dev/howuse/model)

| 模型编码 |弃用日期|指向模型|
| ---- | ---- | ---- |
|chatglm_pro|2024 年 12 月 31 日|glm-4|
|chatglm_std|2024 年 12 月 31 日|glm-3-turbo|
|chatglm_lite|2024 年 12 月 31 日|glm-3-turbo|

In [ ]:
from zhipuai_llm import ZhipuaiLLM
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [ ]:
zhipuai_model = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)

In [ ]:
zhipuai_model.invoke("你好，请你自我介绍一下！")

AIMessage(content='你好！我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', additional_kwargs={}, response_metadata={'time_in_seconds': 1.87}, id='run-4e509a7e-9859-4acb-9418-23245fa5b7a7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 42, 'total_tokens': 53})

## 4.2 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

### 4.2.1 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [23]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

# from langchain.vectorstores.chroma import Chroma
from langchain_chroma import Chroma

从环境变量中加载你的 API_KEY

In [21]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [24]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [25]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1006


我们可以测试一下加载的向量数据库，我们可以通过`as_retriever`方法把向量数据库构造成检索器。我们使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

In [38]:
question = "什么是prompt engineering?"
retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "score_threshold": 0.3}
    )
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


打印一下检索到的内容

In [39]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
-----------------------------------------------------
检索到的第1个内容: 
 ←_←
-----------------------------------------------------
检索到的第2个内容: 
 review for a blender
-----------------------------------------------------


### 4.2.2创建检索链
我们可以使用LangChain的LCEL(LangChain Expression Language, LangChain表达式语言)来构建workflow，LCEL可以支持异步（ainvoke）、流式(stream)、批次处理(batch)等多种运行方式，同时还可以使用LangSmith无缝跟踪。

接下来我们使用刚才定义的retriever定义一个简单的检索链。

In [40]:
from langchain_core.runnables import RunnableLambda
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么？")

'前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n\n\x01本\x03:1.9.9\n发布日期:2023.03\n南  ⽠  书\nPUMPKIN\nB  O  O  K\nDatawhale\n\n←_←'

LCEL中要求所有的组成元素都是`Runnable`类型，前面我们见过的`ChatModel`、`PromptTemplate`等都是继承自`Runnable`类。上方的`retrieval_chain`是由检索器`retriever`及组合器`combiner`组成的，由`|`符号串连，数据从左向右传递，即问题先被`retriever`检索得到检索结果，再被`combiner`进一步处理并输出。

### 4.2.3 创建LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [41]:
from langchain_openai import ChatOpenAI
import os
model = os.environ.get("MODEL_NAME")
llm = ChatOpenAI(model_name=model, temperature=0)

llm.invoke("请你自我介绍一下自己！").content

'<think>\n用户要求我做一个自我介绍。这是一个简单的对话请求，我应该以友好、专业的方式介绍自己。\n\n根据系统提示，我是MiniMax-M2.1，由MiniMax公司开发的AI助手。我应该简洁明了地介绍自己的身份、能力特点。\n\n我会用中文回复，因为用户使用的是中文。\n</think>\n\n你好！我是 MiniMax-M2.1，一个由 MiniMax 公司开发的AI助手。\n\n我具备以下能力：\n\n- **自然语言理解**：能够理解并回应各种中文问题\n- **知识问答**：可以回答涵盖科学、历史、文化、生活等多个领域的问题\n- **写作辅助**：帮助撰写文章、报告、邮件、创意文案等\n- **编程支持**：协助代码编写、调试和技术问题解答\n- **翻译能力**：提供多语言之间的翻译服务\n- **逻辑推理**：帮助分析问题、提供建议和解决方案\n\n我会尽力用清晰、准确的方式与你交流。如果你有任何问题或需要帮助，随时告诉我！'

### 4.2.4 构建检索问答链

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""
# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [45]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# ===== 模块1：文档检索器 =====
def create_retrieval_step():
    """创建文档检索步骤"""
    def retrieve_docs(question):
        retriever = vectordb.as_retriever(
                    search_type="mmr",
                    search_kwargs={"k": 3, "score_threshold": 0.3}
                    )
        return retriever.invoke(question)
    return retrieve_docs

# ===== 模块2：文档合并器 =====
def create_doc_combiner():
    """创建文档合并步骤"""
    def combine_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    return combine_docs

# ===== 模块3：输入构建器 =====
def create_input_builder():
    """创建提示词输入构建器"""
    retrieve = create_retrieval_step()
    combine = create_doc_combiner()
    
    def build_input(question):
        docs = retrieve(question)
        context = combine(docs)
        return {"context": context, "input": question}
    
    return build_input

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""
# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

model = os.environ.get("MODEL_NAME")
llm = ChatOpenAI(model_name=model, temperature=0)

# ===== 模块4：完整问答链 =====
def create_qa_chain():
    """创建完整的RAG问答链"""
    input_builder = create_input_builder()
    
    return (
        RunnablePassthrough()    # 接收输入
        | input_builder         # 构建提示词输入
        | prompt                # 应用提示词
        | llm                   # 调用模型
        | StrOutputParser()      # 解析输出
    )

# 使用
qa_chain = create_qa_chain()


在上边代码中我们把刚才定义的检索链当作子链作为`prompt`的`context`，再使用`RunnablePassthrough`存储用户的问题作为`prompt`的`input`。又因为这两个操作是并行的，所以我们使用`RunnableParallel`来将他们并行运行。

**检索问答链效果测试**

In [46]:
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [47]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
<think>
用户询问什么是南瓜书。根据提供的上下文，南瓜书是Datawhale团队编写的《机器学习》（西瓜书）的补充材料，旨在帮助读者理解西瓜书中较难理解的公式推导细节。它是数学基础较弱的读者在自学时记下的笔记。

我需要用中文回答，简洁明了，最多三句话，并在最后说"谢谢你的提问！"。
</think>

南瓜书是Datawhale团队编写的《机器学习》（西瓜书）配套公式详解手册，主要补充西瓜书中省略的公式推导细节，帮助数学基础较弱的读者更好地理解机器学习内容。简而言之，它是西瓜书的学习笔记和公式解析参考书。谢谢你的提问！


In [48]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后回答 question_2 的结果：")
print(result)

大模型+知识库后回答 question_2 的结果：
<think>
首先，用户说：“使用以下上下文来回答最后的问题。” 然后提供了一段文本，最后的问题是：“Prompt Engineering for Developer是谁写的？”

让我仔细看上下文。上下文是关于Prompt Engineering的说明，但似乎没有直接提到“Prompt Engineering for Developer”是谁写的。用户提到“南瓜书”，但问题具体是关于“Prompt Engineering for Developer”的。

再看上下文内容：它说“总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧...”，但没有提到任何作者名字。

问题本身是：“Prompt Engineering for Developer是谁写的？” 所以我需要基于上下文回答。

但上下文似乎没有提供这个信息。上下文是中文的，标题是“产品说明书”，提到“南瓜书”，但没有说“Prompt Engineering for Developer”的作者。

上下文结尾是“三、英文版”，然后是“产品说明书”，然后是一些版权信息、编委会等，但始终围绕“南瓜书”。

“南瓜书”可能指的是一个具体的书籍或项目。但问题问的是“Prompt Engineering for Developer”，这可能是一个不同的资源。

我需要检查上下文是否包含相关信息。上下文是：

“产品说明书

最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases

编委会

主编：Sm1les、archwalker、jbb0523

编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226

封面设计：构思-Sm1les、创作-林王茂盛

致谢

特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、spareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。

扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”

版权声明

本作品采用知识共享署名-非商

**大模型自己回答的效果**

In [49]:
llm.invoke(question_1).content

'<think>\n用户问的是"什么是南瓜书？"\n\n这个问题需要我明确界定。我需要分析几种可能的"南瓜书"：\n\n1. **《机器学习公式详解》书籍**：这是一本中文机器学习入门书籍，因为封面是南瓜色，所以被俗称为"南瓜书"。这是最可能的含义，因为用户用中文提问，而且"南瓜书"这个称呼在技术圈比较常见。\n\n2. **儿童绘本或童话书**：可能指一些以南瓜为主题的儿童书籍。\n\n3. **烹饪或园艺相关书籍**：关于南瓜种植或食谱的书籍。\n\n考虑到用户用中文提问，且没有特定上下文，最可能指的是那本机器学习的书籍，因为：\n- "南瓜书"这个特定称呼在中文技术社区很有名\n- 这是一本比较受欢迎的机器学习入门书籍\n- 它的特色就是封面是南瓜色，俗称"南瓜书"\n\n我应该主要介绍这本机器学习的书籍，同时简要提及其他可能性。\n</think>\n\n# 南瓜书：机器学习公式详解\n\n"南瓜书"通常指的是《机器学习公式详解》这本书，这是一本中文机器学习领域的入门书籍，因其封面为南瓜色而得名。这本书由谢文睿、秦州编写，人民邮电出版社出版，是西瓜书《机器学习》（周志华著）的配套学习资料。\n\n## 书籍定位与特色\n\n南瓜书的主要特点在于它对机器学习公式的详细推导和解释。许多学习者在阅读西瓜书时常常被复杂的数学公式所困扰，而南瓜书正是为解决这一痛点而生。它不仅给出公式的最终结果，更重要的是展示每一步的推导过程，帮助读者真正理解公式背后的数学原理。\n\n书中内容涵盖了机器学习的基础知识，包括线性模型、决策树、神经网络、支持向量机等核心算法的数学推导。作者采用由浅入深的方式，让读者能够循序渐进地掌握复杂的数学概念。\n\n## 适用人群\n\n南瓜书特别适合以下几类读者：\n- 正在学习机器学习的学生和研究人员\n- 对机器学习数学原理感兴趣的转行者\n- 已经阅读过西瓜书但对公式推导仍有困惑的学习者\n\n## 与西瓜书的关系\n\n西瓜书是周志华教授所著的《机器学习》，因其封面为西瓜图案而得名。南瓜书作为西瓜书的配套参考书，两者相辅相成。南瓜书重点解决西瓜书中公式推导的难点，两者配合使用能够取得更好的学习效果。\n\n总的来说，南瓜书以其详细的公式推导和通俗易懂的讲解风格，成为了机器学习学习者不可或缺的参考书籍之一。'

In [50]:
llm.invoke(question_2).content

'<think>\n用户询问"Prompt Engineering for Developer"是谁写的。这个问题需要我提供准确的信息。\n\n"Prompt Engineering for Developer"应该是指某个具体的文档、书籍或课程。我需要回忆或推断这可能指的是什么。\n\n根据我的知识，"Prompt Engineering for Developer"很可能是指：\n1. 一本关于提示工程的书籍或文档\n2. 某个在线课程或教程\n3. 某个技术文档\n\n实际上，在AI和机器学习领域，有多本关于提示工程的书籍，比如：\n- "Prompt Engineering for Developers"可能是一本技术书籍\n- 也可能是某个在线教育平台的课程\n\n由于用户使用中文提问，我应该用中文回答，并提供我已知的相关信息。\n\n根据我的知识，关于提示工程的知名书籍包括：\n- "The Art of Prompt Engineering"（提示工程的艺术）\n- various guides by different authors\n\n如果这是一个特定的书名或文档，我需要提供准确的作者信息。但考虑到这可能是一般性的询问，我可以提供关于提示工程领域的相关信息。\n\n我应该诚实地说明我的知识局限性，同时提供有用的信息。\n</think>\n\n"Prompt Engineering for Developer"这个书名或文档可能有多个版本，我来为你介绍一下我知道的相关信息：\n\n关于提示工程（Prompt Engineering）的知名著作包括：\n\n**主要相关书籍：**\n- "The Art of Prompt Engineering" by **Fabian Stelzer**\n- "Prompt Engineering Guide" by **DAIR.AI**（这是一个开源社区项目）\n- "Prompt Engineering for GPT-3 and GPT-4" by **Apress出版社出版**\n\n**如果你指的是某个具体的在线课程或文档：**\n可能是某个技术博客、GitHub项目或在线教育平台（如Coursera、Udemy）上的课程。\n\n**建议：**\n如果你能提供更多具体信息（比如出版年份、出版社

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

### 4.2.5 向检索链添加聊天记录

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


**传递聊天记录**

在本节中我们将使用 LangChain 中的`ChatPromptTemplate`，即将先前的对话嵌入到语言模型中，使其具有连续对话的能力。 `ChatPromptTemplate`可以接收聊天消息历史记录，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)
# 制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [ ]:
# 无历史记录
messages = qa_prompt.invoke(
    {
        "input": "南瓜书是什么？",
        "chat_history": [],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


南瓜书是什么？


In [ ]:
# 有历史记录
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "西瓜书是什么？"),
            ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


### 4.2.6 带有信息压缩的检索链

因为我们正在搭建的问答链带有支持多轮对话功能，所以与单轮对话的问答链相比会多面临像上方输出结果的问题，即用户最新的对话语义不全，在使用用户问题查询向量数据库时很难检索到相关信息。像上方的“你可以介绍一下他吗？”，其实是“你可以介绍下周志华老师吗？”的意思。为了解决这个问题我们将采取信息压缩的方式，让llm根据历史记录完善用户的问题。

In [ ]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )
# 构造 压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ])
# 构造检索文档的链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

**支持聊天记录的检索问答链**

这里我们使用之前定义的问答模板即`qa_prompt`构造问答链，另外我们通过`RunnablePassthrough.assign`将中间的查询结果存为`"context"`，将最终结果存为`"answer"`。因为查询结果被存为`"context"`，所以我们整合查询结果的函数`combine_docs`也要做相应的改动。

In [ ]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer

**测试检索问答链**

In [ ]:
# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得')

In [ ]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})

{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己

可以看到，LLM 准确地判断了“它”是什么，代表着我们成功地传递给了它历史信息。另外召回的内容也有着问题的答案，证明我们的信息压缩策略也起到了作用。这种关联前后问题及压缩信息并检索的能力，可大大增强问答系统的连续性和智能水平。

## 4.3 部署知识库助手


我们对知识库和LLM已经有了基本的理解，现在是将它们巧妙地融合并打造成一个富有视觉效果的界面的时候了。这样的界面不仅对操作更加便捷，还能便于与他人分享。

Streamlit 是一种快速便捷的方法，可以直接在 **Python 中通过友好的 Web 界面演示机器学习模型**。在本课程中，我们将学习*如何使用它为生成式人工智能应用程序构建用户界面*。在构建了机器学习模型后，如果你想构建一个 demo 给其他人看，也许是为了获得反馈并推动系统的改进，或者只是因为你觉得这个系统很酷，所以想演示一下：Streamlit 可以让您通过 Python 接口程序快速实现这一目标，而无需编写任何前端、网页或 JavaScript 代码。


### 4.3.1 Streamlit 简介


`Streamlit` 是一个用于快速创建数据应用程序的开源 Python 库。它的设计目标是让数据科学家能够轻松地将数据分析和机器学习模型转化为具有交互性的 Web 应用程序，而无需深入了解 Web 开发。和常规 Web 框架，如 Flask/Django 的不同之处在于，它不需要你去编写任何客户端代码（HTML/CSS/JS），只需要编写普通的 Python 模块，就可以在很短的时间内创建美观并具备高度交互性的界面，从而快速生成数据分析或者机器学习的结果；另一方面，和那些只能通过拖拽生成的工具也不同的是，你仍然具有对代码的完整控制权。

Streamlit 提供了一组简单而强大的基础模块，用于构建数据应用程序：

- st.write()：这是最基本的模块之一，用于在应用程序中呈现文本、图像、表格等内容。

- st.title()、st.header()、st.subheader()：这些模块用于添加标题、子标题和分组标题，以组织应用程序的布局。

- st.text()、st.markdown()：用于添加文本内容，支持 Markdown 语法。

- st.image()：用于添加图像到应用程序中。

- st.dataframe()：用于呈现 Pandas 数据框。

- st.table()：用于呈现简单的数据表格。

- st.pyplot()、st.altair_chart()、st.plotly_chart()：用于呈现 Matplotlib、Altair 或 Plotly 绘制的图表。

- st.selectbox()、st.multiselect()、st.slider()、st.text_input()：用于添加交互式小部件，允许用户在应用程序中进行选择、输入或滑动操作。

- st.button()、st.checkbox()、st.radio()：用于添加按钮、复选框和单选按钮，以触发特定的操作。

这些基础模块使得通过 Streamlit 能够轻松地构建交互式数据应用程序，并且在使用时可以根据需要进行组合和定制，更多内容请查看[官方文档](https://docs.streamlit.io/get-started)

### 4.3.2 构建应用程序

首先，创建一个新的 Python 文件并将其保存 streamlit_app.py在工作目录的根目录中

1. 导入必要的 Python 库。

In [ ]:
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
import sys
sys.path.append("notebook/C3 搭建知识库") # 将父目录放入系统路径中
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain_community.vectorstores import Chroma

2. 定义`get_retriever`函数，该函数返回一个检索器

In [ ]:
def get_retriever():
    # 定义 Embeddings
    embedding = ZhipuAIEmbeddings()
    # 向量数据库持久化路径
    persist_directory = 'data_base/vector_db/chroma'
    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    return vectordb.as_retriever()

3. 定义`combine_docs`函数， 该函数处理检索器返回的文本

In [ ]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"])

4. 定义`get_qa_history_chain`函数，该函数可以返回一个检索问答链

In [ ]:
def get_qa_history_chain():
    retriever = get_retriever()
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    condense_question_system_template = (
        "请根据聊天记录总结用户最近的问题，"
        "如果没有多余的聊天记录则返回用户的问题。"
    )
    condense_question_prompt = ChatPromptTemplate([
            ("system", condense_question_system_template),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ])

    retrieve_docs = RunnableBranch(
        (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
        condense_question_prompt | llm | StrOutputParser() | retriever,
    )

    system_prompt = (
        "你是一个问答任务的助手。 "
        "请使用检索到的上下文片段回答这个问题。 "
        "如果你不知道答案就说不知道。 "
        "请使用简洁的话语回答用户。"
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ]
    )
    qa_chain = (
        RunnablePassthrough().assign(context=combine_docs)
        | qa_prompt
        | llm
        | StrOutputParser()
    )

    qa_history_chain = RunnablePassthrough().assign(
        context = retrieve_docs, 
        ).assign(answer=qa_chain)
    return qa_history_chain

5. 定义`gen_response`函数，它接受检索问答链、用户输入及聊天历史，并以流式返回该链输出

In [ ]:
def gen_response(chain, input, chat_history):
    response = chain.stream({
        "input": input,
        "chat_history": chat_history
    })
    for res in response:
        if "answer" in res.keys():
            yield res["answer"]

6. 定义main函数，该函数制定显示效果与逻辑

In [ ]:
def main():
    st.markdown('### 🦜🔗 动手学大模型应用开发')
    # st.session_state可以存储用户与应用交互期间的状态与数据
    # 存储对话历史
    if "messages" not in st.session_state:
        st.session_state.messages = []
    # 存储检索问答链
    if "qa_history_chain" not in st.session_state:
        st.session_state.qa_history_chain = get_qa_history_chain()
    # 建立容器 高度为500 px
    messages = st.container(height=550)
    # 显示整个对话历史
    for message in st.session_state.messages: # 遍历对话历史
            with messages.chat_message(message[0]): # messages指在容器下显示，chat_message显示用户及ai头像
                st.write(message[1]) # 打印内容
    if prompt := st.chat_input("Say something"):
        # 将用户输入添加到对话历史中
        st.session_state.messages.append(("human", prompt))
        # 显示当前用户输入
        with messages.chat_message("human"):
            st.write(prompt)
        # 生成回复
        answer = gen_response(
            chain=st.session_state.qa_history_chain,
            input=prompt,
            chat_history=st.session_state.messages
        )
        # 流式输出
        with messages.chat_message("ai"):
            output = st.write_stream(answer)
        # 将输出存入st.session_state.messages
        st.session_state.messages.append(("ai", output))

### 4.3.3 部署应用程序  

**本地运行：** `streamlit run "notebook/C4 构建 RAG 应用/streamlit_app.py"`
**远程部署：**要将应用程序部署到 Streamlit Cloud，请执行以下步骤：  
  
1. 为应用程序创建 GitHub 存储库。您的存储库应包含两个文件：  
  
   your-repository/  
   ├── streamlit_app.py  
   └── requirements.txt  
  
2. 转到 [Streamlit Community Cloud](http://share.streamlit.io/)，单击工作区中的`New app`按钮，然后指定存储库、分支和主文件路径。或者，您可以通过选择自定义子域来自定义应用程序的 URL
  
3. 点击`Deploy!`按钮  
  
您的应用程序现在将部署到 Streamlit Community Cloud，并且可以从世界各地访问！ 🌎  

我们的项目部署到这基本完成，为了方便进行演示进行了简化，还有很多可以进一步优化的地方，期待学习者们进行各种魔改！

优化方向：
- 界面中添加上传本地文档，建立向量数据库的功能
- 添加多种LLM 与 embedding方法选择的按钮
- 添加修改参数的按钮
- 更多......